- use 2 modal: T1-pre, T1-post
- batch_size=10
- smaller network
- Data Augmentation (10*positive samples)
- minimize L1 distance + BCE loss
- train-test: 50% -50%

In [9]:
import pickle
import os
import numpy as np
import torch

In [10]:
with open('MRI_Image_TO_EDSS', 'rb') as f:
    MRI_Image_TO_EDSS = pickle.load(f)
print(len(MRI_Image_TO_EDSS))

581


In [11]:
images_T1_post = []
images_T1_pre = []
images_T2 = []
images_flair  = []

labels = [] # np.array([0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0], dtype=np.int64)
MRIs = []

for i in range(len(MRI_Image_TO_EDSS)):
    MRI_Image = str(int(MRI_Image_TO_EDSS[i][0]))
    if MRI_Image not in os.listdir('./img_debias'):
        continue
    if len(os.listdir(os.sep.join(["img_debias", MRI_Image]))) != 5:
        continue
        
    ### T1 Post
    data_path_T1_post = os.sep.join(["img_debias", MRI_Image, "t1_post_reg_des_debis.nii.gz"])
    images_T1_post.append(data_path_T1_post)
    
    ### T1 Pre
    data_path_T1_pre = os.sep.join(["img_debias", MRI_Image, "t1_pre_reg_des_debis.nii.gz"])
    images_T1_pre.append(data_path_T1_pre)
    
    ### T2 
    data_path_T2 = os.sep.join(["img_debias", MRI_Image, "t2_reg_des_debis.nii.gz"])
    images_T2.append(data_path_T2)

    ### flair
    data_path_flair= os.sep.join(["img_debias", MRI_Image, "flair_reg_des_debis.nii.gz"])
    images_flair.append(data_path_flair)
    
    labels.append(int(MRI_Image_TO_EDSS[i][1] > 4.0))
    MRIs.append(MRI_Image)
print(len(images_T1_post), len(images_T1_pre),len(images_T2), len(images_flair), len(labels))
print(labels)
print(len(labels))
# data_path = os.sep.join(["../../", "workspace", "data", "medical", "ixi", "IXI-T1"])

346 346 346 346 346
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0

## Augmentation the training set

In [12]:
##  shuffle first
randinds = np.arange(len(labels))
np.random.seed(42) 
np.random.shuffle(randinds)

MRIs = [MRIs[i] for i in randinds]
labels = [labels[i] for i in randinds]
images_T1_post = [images_T1_post[i] for i in randinds]
images_T1_pre = [images_T1_pre[i] for i in randinds]
images_T2 = [images_T2[i] for i in randinds]
images_flair = [images_flair[i] for i in randinds]

## split train /test
train_frac = 0.5
Ntot = len(labels)

cut = int(train_frac * Ntot)

MRIs_train = MRIs[: cut]
MRIs_test = MRIs[cut:]  ### FUCK!!! wrong here...
labels_train = labels[: cut]
labels_test = labels[cut :]
images_T1_post_train = images_T1_post[: cut]
images_T1_post_test = images_T1_post[cut :]
images_T1_pre_train  = images_T1_pre[: cut]
images_T1_pre_test  = images_T1_pre[cut :]
images_T2_train  = images_T2[: cut]
images_T2_test  = images_T2[cut :]
images_flair_train = images_flair[: cut]
images_flair_test = images_flair[cut :]

## Augment train only
inds = [i for i, x in enumerate(labels_train) if x ==1]
MRIs_train.extend([MRIs_train[i] for i in inds]*10)
labels_train.extend([labels_train[i] for i in inds]*10)
images_T1_post_train.extend([images_T1_post_train[i] for i in inds]*10)
images_T1_pre_train.extend([images_T1_pre_train[i] for i in inds]*10)
images_T2_train.extend([images_T2_train[i] for i in inds]*10)
images_flair_train.extend([images_flair_train[i] for i in inds]*10)

##  shuffle train again
randinds = np.arange(len(labels_train))
np.random.seed(42) 
np.random.shuffle(randinds)
MRIs_train = [MRIs_train[i] for i in randinds]
labels_train = [labels_train[i] for i in randinds]
images_T1_post_train = [images_T1_post_train[i] for i in randinds]
images_T1_pre_train = [images_T1_pre_train[i] for i in randinds]
images_T2_train = [images_T2_train[i] for i in randinds]
images_flair_train = [images_flair_train[i] for i in randinds]

print('====>Train set size:')
print(len(images_T1_post_train), len(images_T1_pre_train),len(images_T2_train), 
      len(images_flair_train), len(labels_train))
print('=======> # positives in Train')
print(np.sum(labels_train))
print('====>Test set size:')
print(len(images_T1_post_test), len(images_T1_pre_test),len(images_T2_test), 
      len(images_flair_test), len(labels_test))
print('=======> # positives in Test')
print(np.sum(labels_test))


====>Train set size:
353 353 353 353 353
=======> # positives in Train
198
====>Test set size:
173 173 173 173 173
=======> # positives in Test
15


In [13]:
len(set(MRIs_train)), len(set(MRIs_test)), \
len(set(MRIs_train).intersection(MRIs_test))

(173, 173, 0)

## set label weight  in train

In [14]:
num_classes = len(set(labels_train))
class_count = [np.sum(np.array(labels_train) == i ) for i in range(num_classes)]
print(num_classes, class_count)
class_weights = 1./torch.tensor(class_count, dtype=torch.float) 
print(class_weights)

2 [155, 198]
tensor([0.0065, 0.0051])


In [15]:
np.sum(labels_train), len(labels_train)

(198, 353)

# Util

In [16]:
from sklearn.metrics import roc_curve, confusion_matrix , auc, precision_recall_curve, average_precision_score
from sklearn import metrics

def print_metrics(y_true, y_pred):
    import matplotlib.pyplot as plt
    
    false_positive_rate, recall, thresholds = roc_curve(y_true, y_pred)
    roc_auc = auc(false_positive_rate, recall)
    auprc = average_precision_score(y_true, y_pred)
    print('AUC: ',roc_auc, "AUPRC: ", auprc)


    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred)

    # calculate the g-mean for each threshold
    gmeans = np.sqrt(tpr * (1-fpr))
    ix = np.argmax(gmeans)
    print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))


    tn, fp, fn, tp = confusion_matrix(y_true, y_pred > thresholds[ix]).ravel()
    print('TN: ', tn, ", FP: ",fp, ", FN:", fn, ", TP:", tp)
    print("==> Sensitivity (Recall, TPR): %.3f"%(tp/(tp+fn)))
    print("==> Specifity: %.3f"%(tn/(tn+fp)))
    print("==> Positive Predictive Value (PPV) (Precision): %.3f"%(tp / (tp + fp)))
    print("==> Negative Predictive Value (NPV): %.3f"%(tn / (tn + fn)))
    print("==> Accuracy: %.3f"%((tp+tn)/(tn+ fp+ fn+tp)))
    print("==> F1 score: %.3f"%((2*tp)/(2*tp + fp + fn)))
    
    ns_probs = [0 for _ in range(len(y_true))]
    
    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(y_true, ns_probs)
    lr_fpr, lr_tpr, _ = roc_curve(y_true, y_pred)
    # plot the roc curve for the model
    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill, AUC = %0.2f' % 0.5)
    plt.plot(lr_fpr, lr_tpr, marker='.', label = 'Our model: AUC = %0.2f' % roc_auc)
    # axis labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    # show the legend
    plt.legend()
    # show the plot
    plt.show()
    
    lr_precision, lr_recall, _ = precision_recall_curve(y_true, y_pred)
    lr_auc = auc(lr_recall, lr_precision)
    # summarize scores
#     print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))
    # plot the precision-recall curves
    y_true = np.array(y_true)
    no_skill = len(y_true[y_true==1]) / len(y_true)
    plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No skill, AUPRC = %0.2f' % no_skill)
    plt.plot(lr_recall, lr_precision, marker='.',label = 'Our model: AUPRC = %0.2f' % auprc)
    # axis labels
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    # show the legend
    plt.legend(loc = 'upper right')
    # show the plot
    plt.show()


# model

In [17]:
import torch.nn as nn
from typing import Callable, Sequence, Type, Union
import torch
from collections import OrderedDict
from monai.networks.layers.factories import Conv, Dropout, Pool
from monai.networks.layers.utils import get_act_layer, get_norm_layer
from monai.utils.module import look_up_option

class _Transition(nn.Sequential):
    def __init__(
        self,
        spatial_dims: int,
        in_channels: int,
        out_channels: int,
        act: Union[str, tuple] = ("relu", {"inplace": False}),
        norm: Union[str, tuple] = "batch",
    ) -> None:
        """
        Args:
            spatial_dims: number of spatial dimensions of the input image.
            in_channels: number of the input channel.
            out_channels: number of the output classes.
            act: activation type and arguments. Defaults to relu.
            norm: feature normalization type and arguments. Defaults to batch norm.
        """
        super().__init__()

        conv_type: Callable = Conv[Conv.CONV, spatial_dims]
        pool_type: Callable = Pool[Pool.AVG, spatial_dims]

        self.add_module("norm", get_norm_layer(name=norm, spatial_dims=spatial_dims, channels=in_channels))
        self.add_module("relu", get_act_layer(name=act))
        self.add_module("conv", conv_type(in_channels, out_channels, kernel_size=1, bias=False))
        self.add_module("pool", pool_type(kernel_size=2, stride=2))

class _DenseLayer(nn.Module):
    def __init__(
        self,
        spatial_dims: int,
        in_channels: int,
        growth_rate: int,
        bn_size: int,
        dropout_prob: float,
        act: Union[str, tuple] = ("relu", {"inplace": False}),
        norm: Union[str, tuple] = "batch",
    ) -> None:
        """
        Args:
            spatial_dims: number of spatial dimensions of the input image.
            in_channels: number of the input channel.
            growth_rate: how many filters to add each layer (k in paper).
            bn_size: multiplicative factor for number of bottle neck layers.
                (i.e. bn_size * k features in the bottleneck layer)
            dropout_prob: dropout rate after each dense layer.
            act: activation type and arguments. Defaults to relu.
            norm: feature normalization type and arguments. Defaults to batch norm.
        """
        super().__init__()

        out_channels = bn_size * growth_rate
        conv_type: Callable = Conv[Conv.CONV, spatial_dims]
        dropout_type: Callable = Dropout[Dropout.DROPOUT, spatial_dims]

        self.layers = nn.Sequential()

        self.layers.add_module("norm1", get_norm_layer(name=norm, spatial_dims=spatial_dims, channels=in_channels))
        self.layers.add_module("relu1", get_act_layer(name=act))
        self.layers.add_module("conv1", conv_type(in_channels, out_channels, kernel_size=1, bias=False))

        self.layers.add_module("norm2", get_norm_layer(name=norm, spatial_dims=spatial_dims, channels=out_channels))
        self.layers.add_module("relu2", get_act_layer(name=act))
        self.layers.add_module("conv2", conv_type(out_channels, growth_rate, kernel_size=3, padding=1, bias=False))

        if dropout_prob > 0:
            self.layers.add_module("dropout", dropout_type(dropout_prob))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        new_features = self.layers(x)
        return torch.cat([x, new_features], 1)
    
class _DenseBlock(nn.Sequential):
    def __init__(
        self,
        spatial_dims: int,
        layers: int,
        in_channels: int,
        bn_size: int,
        growth_rate: int,
        dropout_prob: float,
        act: Union[str, tuple] = ("relu", {"inplace": False}),
        norm: Union[str, tuple] = "batch",
    ) -> None:
        """
        Args:
            spatial_dims: number of spatial dimensions of the input image.
            layers: number of layers in the block.
            in_channels: number of the input channel.
            bn_size: multiplicative factor for number of bottle neck layers.
                (i.e. bn_size * k features in the bottleneck layer)
            growth_rate: how many filters to add each layer (k in paper).
            dropout_prob: dropout rate after each dense layer.
            act: activation type and arguments. Defaults to relu.
            norm: feature normalization type and arguments. Defaults to batch norm.
        """
        super().__init__()
        for i in range(layers):
            layer = _DenseLayer(spatial_dims, in_channels, growth_rate, bn_size, dropout_prob, act=act, norm=norm)
            in_channels += growth_rate
            self.add_module("denselayer%d" % (i + 1), layer)
            
class DenseNet(nn.Module):
    """
    Densenet based on: `Densely Connected Convolutional Networks <https://arxiv.org/pdf/1608.06993.pdf>`_.
    Adapted from PyTorch Hub 2D version: https://pytorch.org/vision/stable/models.html#id16.
    Args:
        spatial_dims: number of spatial dimensions of the input image.
        in_channels: number of the input channel.
        out_channels: number of the output classes.
        init_features: number of filters in the first convolution layer.
        growth_rate: how many filters to add each layer (k in paper).
        block_config: how many layers in each pooling block.
        bn_size: multiplicative factor for number of bottle neck layers.
            (i.e. bn_size * k features in the bottleneck layer)
        act: activation type and arguments. Defaults to relu.
        norm: feature normalization type and arguments. Defaults to batch norm.
        dropout_prob: dropout rate after each dense layer.
    """

    def __init__(
        self,
        spatial_dims: int,
        in_channels: int,
        out_channels: int,
        init_features: int = 64,
        growth_rate: int = 32,
        block_config: Sequence[int] = (6, 12, 24, 16),
        bn_size: int = 4,
        act: Union[str, tuple] = ("relu", {"inplace": False}),
        norm: Union[str, tuple] = "batch",
        dropout_prob: float = 0.0,
    ) -> None:

        super().__init__()

        conv_type: Type[Union[nn.Conv1d, nn.Conv2d, nn.Conv3d]] = Conv[Conv.CONV, spatial_dims]
        pool_type: Type[Union[nn.MaxPool1d, nn.MaxPool2d, nn.MaxPool3d]] = Pool[Pool.MAX, spatial_dims]
        avg_pool_type: Type[Union[nn.AdaptiveAvgPool1d, nn.AdaptiveAvgPool2d, nn.AdaptiveAvgPool3d]] = Pool[
            Pool.ADAPTIVEAVG, spatial_dims
        ]

        self.features = nn.Sequential(
            OrderedDict(
                [
                    ("conv0", conv_type(in_channels, init_features, kernel_size=7, stride=2, padding=3, bias=False)),
                    ("norm0", get_norm_layer(name=norm, spatial_dims=spatial_dims, channels=init_features)),
                    ("relu0", get_act_layer(name=act)),
                    ("pool0", pool_type(kernel_size=3, stride=2, padding=1)),
                ]
            )
        )

        in_channels = init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(
                spatial_dims=spatial_dims,
                layers=num_layers,
                in_channels=in_channels,
                bn_size=bn_size,
                growth_rate=growth_rate,
                dropout_prob=dropout_prob,
                act=act,
                norm=norm,
            )
            self.features.add_module(f"denseblock{i + 1}", block)
            in_channels += num_layers * growth_rate
#             print('in_channels is ', in_channels)
            if i == len(block_config) - 1:
                self.features.add_module(
                    "norm5", get_norm_layer(name=norm, spatial_dims=spatial_dims, channels=in_channels)
                )
            else:
                _out_channels = in_channels // 2
                trans = _Transition(
                    spatial_dims, in_channels=in_channels, out_channels=_out_channels, act=act, norm=norm
                )
                self.features.add_module(f"transition{i + 1}", trans)
                in_channels = _out_channels

        # pooling and classification
        self.class_layers = nn.Sequential(
            OrderedDict(
                [
                    ("relu", get_act_layer(name=act)),
                    ("pool", avg_pool_type(1)),
                    ("flatten", nn.Flatten(1)),
                    ("out", nn.Linear(in_channels, out_channels)),
                ]
            )
        )
        ### get them out
        self.class_layers_1 = get_act_layer(name=act)
        self.class_layers_2 = avg_pool_type(1)
        self.class_layers_3 = nn.Flatten(1)
        MODAL = 1
        self.class_layers_4 = nn.Linear(in_channels * MODAL, out_channels)
        
        self.sigmoid = nn.Sigmoid()

        for m in self.modules():
            if isinstance(m, conv_type):
                nn.init.kaiming_normal_(torch.as_tensor(m.weight))
            elif isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d)):
                nn.init.constant_(torch.as_tensor(m.weight), 1)
                nn.init.constant_(torch.as_tensor(m.bias), 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(torch.as_tensor(m.bias), 0)

        
    def forward(self, x_flair: torch.Tensor
                   ) -> torch.Tensor:   # [2, 1, 96, 96, 96]
        ####  ==================== flair ==================================
        x_flair = self.features(x_flair)    # [2, 1024, 3, 3, 3]
        x_flair = self.class_layers_1(x_flair)  # output: [2, 1024, 3, 3, 3]
        x_flair = self.class_layers_2(x_flair) # output: [2, 1024, 1, 1, 1]
        x_flair = self.class_layers_3(x_flair) # output: [2, 1024]
        x_join = x_flair
        linear_y = self.class_layers_4(x_join) # output: [2, 2]
        out = self.sigmoid(linear_y)
        return x_join, linear_y , out

    
class DenseNet121(DenseNet):
    """DenseNet121 with optional pretrained support when `spatial_dims` is 2."""

    def __init__(
        self,
        init_features: int = 32, # 64,
        growth_rate: int = 16, # 32,
        block_config: Sequence[int] = (3,6,12,8),#(6, 12, 24, 16),
        pretrained: bool = False,
        progress: bool = True,
        **kwargs,
    ) -> None:
        super().__init__(init_features=init_features, growth_rate=growth_rate, block_config=block_config, **kwargs)
        if pretrained:
            if kwargs["spatial_dims"] > 2:
                raise NotImplementedError(
                    "Parameter `spatial_dims` is > 2 ; currently PyTorch Hub does not"
                    "provide pretrained models for more than two spatial dimensions."
                )
            _load_state_dict(self, "densenet121", progress)

# Train

In [19]:
import logging
import os
import sys

import numpy as np
import torch
from torch.utils.data import DataLoader
# from torch.utils.tensorboard import SummaryWriter

import monai
from monai.data import ImageDataset
from monai.transforms import AddChannel, Compose, RandRotate, Resize, ScaleIntensity, EnsureType



# Define transforms
train_transforms = Compose([ScaleIntensity(), AddChannel(), Resize((256,256,44)), RandRotate(range_x=0.02, range_y=0.02, range_z=0.02, prob=0.5), EnsureType()])
val_transforms = Compose([ScaleIntensity(), AddChannel(), Resize((256,256,44)), EnsureType()])

#### ======================================= flair  ===============================================
# Define image dataset, data loader
check_ds_flair = ImageDataset(image_files=images_flair_train, labels=labels_train, transform=train_transforms)
check_loader_flair= DataLoader(check_ds_flair, batch_size=10, num_workers=2)# , pin_memory=torch.cuda.is_available())
# create a training data loader
train_ds_flair = ImageDataset(image_files=images_flair_train, labels=labels_train, transform=train_transforms)
train_loader_flair = DataLoader(train_ds_flair, batch_size=10, shuffle=False, num_workers=2)#, pin_memory=torch.cuda.is_available())
#create a validation data loader
val_ds_flair = ImageDataset(image_files=images_flair_test, labels=labels_test, transform=val_transforms)
val_loader_flair = DataLoader(val_ds_flair, batch_size=10, num_workers=2)#, pin_memory=torch.cuda.is_available())


# Create DenseNet121, CrossEntropyLoss and Adam optimizer
device = torch.device(13 if torch.cuda.is_available() else "cpu")
model = DenseNet121(spatial_dims=3, in_channels=1, out_channels=2).to(device)
loss_function = torch.nn.BCEWithLogitsLoss(weight=class_weights.to(device))
optimizer = torch.optim.Adam(model.parameters(), 1e-5)

triplet_loss = \
    nn.TripletMarginWithDistanceLoss(distance_function=nn.PairwiseDistance(), margin=1.5)



# start a typical PyTorch training
EPOCH = 400
val_interval = 2
best_metric = -1
epoch_loss_values = list()
metric_values = list()
best_AUC=0

for epoch in range(EPOCH):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{EPOCH}")
    model.train()
    epoch_loss = 0
    correct = 0
    step = 0
    total = 0
    
    iter_flair = iter(train_loader_flair)
           
    while step < len(train_loader_flair):
        step += 1
        inputs_flair, labels = next(iter_flair)
        inputs_flair = inputs_flair.to(device)
        one_hot_label = np.eye(2)[np.array(labels,dtype="int")]
        one_hot_label = torch.tensor(one_hot_label)
        optimizer.zero_grad()
        joint_embedding, out, predict = model(inputs_flair)
        one_hot_label = one_hot_label.type_as(out).to(device)
        loss_1 = loss_function(out, one_hot_label)

        anchor = 0 
        positive = [i for i, x in enumerate(labels) if x == 1]
        negative = [i for i, x in enumerate(labels) if x == 0]
        if not positive or not negative:
            continue
        loss = loss_1
        for i in positive:
            for j in negative:
                loss += triplet_loss(0*joint_embedding[0,:], joint_embedding[i,:], joint_embedding[j,:])
        loss.backward()
        _, predicted = torch.max(predict.detach().cpu(), 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_len = len(train_ds_flair) // train_loader_flair.batch_size
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()

        with torch.no_grad():
            num_correct = 0.0
            metric_count = 0
            pred_list = []
            true_list = []
            predicted_list=[]
            prediction_probablity=[]
            label_list = []
            val_total=0
            val_correct = 0
            
            iter_flair = iter(val_loader_flair)
            step = 0
            while step < len(val_loader_flair):
                step += 1
                val_images_flair, val_labels = next(iter_flair)
                val_images_flair = val_images_flair.to(device)
                _, out,predict = model(val_images_flair)
                _, predicted = torch.max(predict.detach().cpu(), 1)
                predicted_list.append(predicted.cpu().numpy())
                predicted_2 = predict.detach().cpu().numpy()
                prediction_prob = predicted_2[:,1].tolist()
                prediction_probablity.extend(prediction_prob)
                label_list.extend(val_labels.cpu().numpy().tolist())
                val_total += val_labels.size(0)
                val_correct += (predicted == val_labels).sum().item()
            
            
            Accuracy = val_correct/val_total
            y=np.array(label_list)
            false_positive_rate, recall, thresholds = roc_curve(y.flatten(), np.array(prediction_probablity).flatten())
            roc_auc = auc(false_positive_rate, recall)
            auprc = average_precision_score(y.flatten(), np.array(prediction_probablity).flatten())
            
            if roc_auc > best_AUC:
                best_AUC = roc_auc
                torch.save(model.state_dict(), "Models_saved-copy3-flair-Medcam/State_checkpoints_{}.thr".format(epoch))
                torch.save(model, "Models_saved-copy3-flair-Medcam/Model_checkpoints_{}.thr".format(epoch))
            print("AUC: {} , Accuracy: {}, AUPRC: {}".format(roc_auc, Accuracy, auprc))

print(f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")

----------
epoch 1/400
epoch 1 average loss: 37.6053
----------
epoch 2/400
epoch 2 average loss: 27.0376
AUC: 0.49999999999999994 , Accuracy: 0.08670520231213873, AUPRC: 0.08919763425545615
----------
epoch 3/400
epoch 3 average loss: 20.0756
----------
epoch 4/400
epoch 4 average loss: 15.5619
AUC: 0.36244725738396627 , Accuracy: 0.08670520231213873, AUPRC: 0.06877537065368718
----------
epoch 5/400
epoch 5 average loss: 12.6485
----------
epoch 6/400
epoch 6 average loss: 10.3084
AUC: 0.3611814345991561 , Accuracy: 0.08670520231213873, AUPRC: 0.06817744584890936
----------
epoch 7/400
epoch 7 average loss: 8.2573
----------
epoch 8/400
epoch 8 average loss: 6.5545
AUC: 0.3810126582278481 , Accuracy: 0.08670520231213873, AUPRC: 0.070301040170998
----------
epoch 9/400
epoch 9 average loss: 5.0859
----------
epoch 10/400
epoch 10 average loss: 3.8293
AUC: 0.43333333333333335 , Accuracy: 0.10404624277456648, AUPRC: 0.0769346306866216
----------
epoch 11/400
epoch 11 average loss: 2.876

epoch 87 average loss: 0.0034
----------
epoch 88/400
epoch 88 average loss: 0.0034
AUC: 0.839240506329114 , Accuracy: 0.9075144508670521, AUPRC: 0.3545828040768856
----------
epoch 89/400
epoch 89 average loss: 0.0034
----------
epoch 90/400
epoch 90 average loss: 0.0034
AUC: 0.8383966244725739 , Accuracy: 0.9075144508670521, AUPRC: 0.3527898693754052
----------
epoch 91/400
epoch 91 average loss: 0.0034
----------
epoch 92/400
epoch 92 average loss: 0.0034
AUC: 0.8379746835443038 , Accuracy: 0.9017341040462428, AUPRC: 0.35210021420299137
----------
epoch 93/400
epoch 93 average loss: 0.0034
----------
epoch 94/400
epoch 94 average loss: 0.0034
AUC: 0.8371308016877638 , Accuracy: 0.9017341040462428, AUPRC: 0.3511004429835908
----------
epoch 95/400
epoch 95 average loss: 0.0033
----------
epoch 96/400
epoch 96 average loss: 0.0033
AUC: 0.8371308016877638 , Accuracy: 0.9017341040462428, AUPRC: 0.35062632322816173
----------
epoch 97/400
epoch 97 average loss: 0.0033
----------
epoch 98

AUC: 0.810970464135021 , Accuracy: 0.8959537572254336, AUPRC: 0.25529527015143905
----------
epoch 173/400
epoch 173 average loss: 0.0027
----------
epoch 174/400
epoch 174 average loss: 0.0027
AUC: 0.8105485232067511 , Accuracy: 0.8959537572254336, AUPRC: 0.25490812342282887
----------
epoch 175/400
epoch 175 average loss: 0.0027
----------
epoch 176/400
epoch 176 average loss: 0.0027
AUC: 0.8105485232067511 , Accuracy: 0.8959537572254336, AUPRC: 0.25490812342282887
----------
epoch 177/400
epoch 177 average loss: 0.0027
----------
epoch 178/400
epoch 178 average loss: 0.0027
AUC: 0.810126582278481 , Accuracy: 0.8959537572254336, AUPRC: 0.2540627127948095
----------
epoch 179/400
epoch 179 average loss: 0.0027
----------
epoch 180/400
epoch 180 average loss: 0.0027
AUC: 0.8105485232067511 , Accuracy: 0.8959537572254336, AUPRC: 0.2559145646466614
----------
epoch 181/400
epoch 181 average loss: 0.0027
----------
epoch 182/400
epoch 182 average loss: 0.0026
AUC: 0.8105485232067511 , Acc

epoch 257 average loss: 0.0022
----------
epoch 258/400
epoch 258 average loss: 0.0021
AUC: 0.809282700421941 , Accuracy: 0.9017341040462428, AUPRC: 0.2678090969921948
----------
epoch 259/400
epoch 259 average loss: 0.0021
----------
epoch 260/400
epoch 260 average loss: 0.0021
AUC: 0.8097046413502109 , Accuracy: 0.9017341040462428, AUPRC: 0.26799689041942487
----------
epoch 261/400
epoch 261 average loss: 0.0021
----------
epoch 262/400
epoch 262 average loss: 0.0021
AUC: 0.8097046413502109 , Accuracy: 0.8959537572254336, AUPRC: 0.2674911552153578
----------
epoch 263/400
epoch 263 average loss: 0.0021
----------
epoch 264/400
epoch 264 average loss: 0.0022
AUC: 0.7278481012658227 , Accuracy: 0.9017341040462428, AUPRC: 0.2302113987958467
----------
epoch 265/400
epoch 265 average loss: 0.0406
----------
epoch 266/400
epoch 266 average loss: 0.1346
AUC: 0.7565400843881857 , Accuracy: 0.8901734104046243, AUPRC: 0.23010688693596568
----------
epoch 267/400
epoch 267 average loss: 0.128

epoch 342 average loss: 0.0017
AUC: 0.7624472573839662 , Accuracy: 0.8959537572254336, AUPRC: 0.2545584699461962
----------
epoch 343/400
epoch 343 average loss: 0.0016
----------
epoch 344/400
epoch 344 average loss: 0.0016
AUC: 0.7641350210970463 , Accuracy: 0.8959537572254336, AUPRC: 0.2553203432110787
----------
epoch 345/400
epoch 345 average loss: 0.0016
----------
epoch 346/400
epoch 346 average loss: 0.0016
AUC: 0.7654008438818565 , Accuracy: 0.8959537572254336, AUPRC: 0.2574148731537099
----------
epoch 347/400
epoch 347 average loss: 0.0016
----------
epoch 348/400
epoch 348 average loss: 0.0016
AUC: 0.7670886075949367 , Accuracy: 0.8959537572254336, AUPRC: 0.25964733509964233
----------
epoch 349/400
epoch 349 average loss: 0.0016
----------
epoch 350/400
epoch 350 average loss: 0.0016
AUC: 0.7670886075949367 , Accuracy: 0.8959537572254336, AUPRC: 0.2596400783066713
----------
epoch 351/400
epoch 351 average loss: 0.0016
----------
epoch 352/400
epoch 352 average loss: 0.001

NameError: name 'best_metric_epoch' is not defined

In [20]:
best_AUC

0.8552742616033755

In [ ]:
#### ======================================= flair ===============================================
# Define image dataset, data loader
check_ds_flair = ImageDataset(image_files=images_flair, labels=labels, transform=train_transforms)
check_loader_flair = DataLoader(check_ds_flair, batch_size=1, num_workers=2)# , pin_memory=torch.cuda.is_available())
im, label = monai.utils.misc.first(check_loader_flair)
# create a training data loader
train_ds_flair = ImageDataset(image_files=images_flair[:train_frac], labels=labels[:train_frac], transform=train_transforms)
train_loader_flair = DataLoader(train_ds_flair, batch_size=1, shuffle=False, num_workers=2)#, pin_memory=torch.cuda.is_available())
#create a validation data loader
val_ds_flair = ImageDataset(image_files=images_flair[train_frac:], labels=labels[train_frac:], transform=val_transforms)
val_loader_flair = DataLoader(val_ds_flair, batch_size=1, num_workers=2)#, pin_memory=torch.cuda.is_available())


# Import M3d-CAM
from medcam import medcam

model.load_state_dict(torch.load("Models_saved-copy3-flair-Medcam/State_checkpoints_{}.thr".format()))
# Inject model with M3d-CAM
model_medCAM = medcam.inject(model, output_dir="flair_attention_maps", save_maps=True)
                 
model_medCAM.eval()


with torch.no_grad():
    num_correct = 0.0
    metric_count = 0
    pred_list = []
    true_list = []
    predicted_list=[]
    prediction_probablity=[]
    label_list = []
    val_total=0
    val_correct = 0

    iter_flair = iter(val_loader_flair)
    
    step = 0
    while step < len(val_loader_flair):
        step += 1
        val_images_flair, val_labels = next(iter_flair)
        val_images_flair = val_images_flair.to(device)
        _= model_medCAM(val_images_flair)
        print(step)

In [ ]:
print(val_images_flair.shape)
import nibabel as nib
test_mask=nib.load('flair_attention_maps/features.transition3.conv/attention_map_10_0_0.nii.gz')# .get_data()
test_mask.shape
